Separately run referrals and claims queries.
This file imports the results of those queries, cleans and standardizes each, and merges them for analysis.

In addition, we import clinical decision files and use it to "override" decisions that were made by analyzing the ROI for specialty & cpt code combination.

Finally, several outputs are prepared for reporting purposes, including the calculation of a projected AA approval rate.

In [129]:
## Import required packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", 100)
import pyodbc
import os

## Import Data

#### Import data directly from SQL databases

Use referrals.sql, claims.sql, referrals_new.sql, cpt_desc.sql (saved in the same folder as this script) to pull data from relevants servers and databases.

In [130]:
with open('referrals_quarterly.sql', 'r') as myfile:
    cpts_sql_str=myfile.read().replace('\n', ' ')

cnxn_cpts = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

cpts = pd.read_sql(cpts_sql_str, cnxn_cpts)

cnxn_cpts.close()

In [131]:
with open('claims_quarterly.sql', 'r') as myfile:
    claims_sql_str=myfile.read().replace('\n', ' ')

cnxn_claims = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=NATIONAL_ANALYTICS')

claims = pd.read_sql(claims_sql_str, cnxn_claims)

cnxn_claims.close()

In [132]:
with open('cpt_desc.sql', 'r') as myfile:
    cpt_desc_sql_str=myfile.read().replace('\n', ' ')

cnxn_cpt_desc = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

cpt_desc = pd.read_sql(cpt_desc_sql_str, cnxn_cpt_desc)

cnxn_cpt_desc.close()

In [133]:
clinical_decisions_current = pd.read_excel('../data/AADictionary_Final_52919.xlsx', sheet_name='Decisions')

In [134]:
#clinical_decisions_20190610 = pd.read_csv('../data/clinical_decision_20190610.csv')

In [135]:
clinical_decisions_current = clinical_decisions_current[['Specialty', 'CPT_Code', 'is_PPL', 'Decision_Source',
       'Decision_Maker', 'Reason', 'Decision_Date', 'Decision']]

## Clean Referrals

A list of all specialties is used repeatedly in the code to loop through specialties and perform certain tasks.

In [136]:
# A list of all specialties is used repeatedly in the code to loop through 
# specialties and perform certain tasks.
list_o_specs = cpts['Specialty'].unique().tolist()

In [137]:
## Update date fields to conform to python datetime
cpts['Date_Decision'] = pd.to_datetime(cpts['Date_Decision'])
cpts['Date_Received'] = pd.to_datetime(cpts['Date_Received'])

In [138]:
## Some specialties need to be broken into Senior and non-commercial 
## such that we can auto-approve for specific lines of business
new_lob = {'COMMERCIAL': '_not_sen',
               'SENIOR': '_senior',
          'MEDI-CAL': '_not_sen'}
cpts['LOB'] = cpts['LOB'].replace(new_lob)

In [139]:
## Define a list of specialties that will be broken out into LOB for the purposes of AA
specs_w_lob_distinct = ['RADIOLOGY']

In [140]:
## Loop through specialties that should be broken by LOB and update the specialty column
for spec in list_o_specs:
    if spec in specs_w_lob_distinct:
        cpts['Specialty'] = np.where(cpts['Specialty']==spec, cpts['Specialty']+cpts['LOB'], cpts['Specialty'])
    

In [141]:
# Now that some specialties are broken into senior and non-senior, need to recreate the list_o_specs
list_o_specs = cpts['Specialty'].unique().tolist()

In [142]:
## Append specialties manually that only appear in CCT data
#list_o_specs.append('HIV/AIDS SPECIALIST')

In [143]:
## flag retro statuses with 1 and 0 so they can be removed 
retro_conditions = [
 (cpts['status_name'] == 'APPROVED - RETRO REVIEW') |
 (cpts['status_name'] == 'DENIED - RETRO REVIEW') |
 (cpts['status_name'] == 'APPROVED - COB RETRO') |
 (cpts['status_name'] == 'PENDING - RETRO REVIEW') 
  ]

choices = [1]
cpts['is_retro'] = np.select(retro_conditions, choices, default=0)

## remove retros from list and drop 'is_retro' as it is no longer needed
cpts = cpts[cpts['is_retro']==0]
cpts.drop(columns='is_retro', inplace=True)

In [144]:
## Claims data doesn't come with UNITS & we need to count the number of times a cpt code appears
## Here we re-write UNITS to 1.
cpts['UNITS'] = 1

In [145]:
## Create an auto_approve flag
cpts['is_autoapp'] = np.where(cpts['status_name']=='APPROVED - AUTO', 1, 0)

In [146]:
## If PPL field is null, assume the referral was not PPL
cpts.PPL.fillna("N", inplace=True)

In [147]:
## Create a PPL flag
cpts['is_PPL'] = np.where(cpts['PPL']=='Y', 1, 0)

In [148]:
## define list of status that should be considered denials
den_conditions = [
 (cpts['status_name'] == 'DENIED - CM') |
 (cpts['status_name'] == 'DENIED - BENEFIT CARVE OUT') |
 (cpts['status_name'] == 'DENIED - NOT A COVERED BENEFIT') |
 (cpts['status_name'] == 'DENIED - APPEAL') |
 (cpts['status_name'] == 'DENIED - CLINICAL TRIAL/EXP/INV') |
 (cpts['status_name'] == 'DENIED - TRANSPLANT') |
 (cpts['status_name'] == 'DENIED - MD') |
 (cpts['status_name'] == 'DENIED - CM/MD') |
 (cpts['status_name'] == 'DENIED - REDIRECT OSVN') |
 (cpts['status_name'] == 'DENIED - TICKLER')
  ]

## Create a denial flag
choices = [1]
cpts['is_den'] = np.select(den_conditions, choices, default=0)

In [149]:
## Create a "approved" flag
cpts['is_app'] = np.where(cpts['status_cat']=='APPROVED', 1, 0)

## Create claims_sum

In [150]:
## Some Specialty/CPT code combos appear in referrals data but not in claims data. 
## In those cases, we look in the claims data across all specialties to find an average
## Cost to be applied for that CPT Code.
claims_sum = claims.groupby(['CPT_Code'], as_index=False).agg({'avg_hcp_cost': 'mean'})

## Add variable to designate last 3 vs. prior 3

In [151]:
## Split CPTs data into two parts, one for PRIOR_3 and one for LAST_3

In [152]:
cpts['last3'] = np.where(cpts['Date_Received'] >= '2019-04-28', 1, 0)

## Create cpts_manual

In [153]:
## This is a cpt_code level list of all manually reviewed referrals

In [154]:
cpts_manual = cpts[cpts['is_autoapp']==0]

In [155]:
cpts_manual.pivot_table(values='HCP_CONNECT_AUTH_NUMBER', index=['is_autoapp', 'is_den'], aggfunc='count', margins=True)

HCP_CONNECT_AUTH_NUMBER
is_autoapp is_den                         
0          0                        789321
           1                         19772
All                                 809093

In [156]:
## Enter the total GA for the department for the year, divide by 2 since we're only looking at 1/2 of the year
dept_ga = 6500000 / 2

In [157]:
## Calculate Cost per manually reviewed CPT code

ga_cpt = dept_ga / cpts[cpts['is_autoapp']==0].shape[0]

In [158]:
ga_cpt

3.975335793556531

In [159]:
## find the count of manually reviewed cpt codes from each specialty, cpt_code combo 
cpts_manual = cpts_manual.groupby(['last3', 'Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS' : 'count',
    'is_den': 'mean'
})

In [160]:
## To differentiate the count of all units from manual units as we use both in a single
## file later
cpts_manual.rename(index=str, columns={'UNITS': 'UNITS_man'}, inplace=True)

In [161]:
## Calculate the total cost of review any Specialty/cpt_code pair.
cpts_manual['cost_to_review'] = cpts_manual['UNITS_man']*ga_cpt

## Create cptssum

In [162]:
## find the count, auto-approval rate, and denial rate from each specialty, cpt_code pair
cpts2 = cpts.groupby(['last3', 'Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS': 'count',
    'is_autoapp': 'mean',
})

In [163]:
## Merge the cost to review (from CPT_manual) into cptssum
cpts3 = pd.merge(cpts2, cpts_manual, on=['last3', 'Specialty', 'CPT_Code', 'is_PPL'], how='left')

In [164]:
## Merge average cost of a cpt code (from claims) into the referrals data
cpts4 = pd.merge(cpts3, claims, on=['Specialty', 'CPT_Code'], how='left')

In [165]:
## For spec/cpt combos that don't have claims data associated, 
## use the average of that cpt across specialties
## NOTE: the average is not weighted, i.e. each specialties's average contributes equally to
## the applied average.
cpts_w_claims_fin = pd.merge(cpts4, claims_sum, on='CPT_Code', how='left')

In [166]:
## If there is no average cost from claims at the spec/cpt pair level, fill it with the 
## average cost for the cpt ACROSS ALL SPECIALTIES
cpts_w_claims_fin['avg_hcp_cost_x'] = np.where(cpts_w_claims_fin['avg_hcp_cost_x'].isnull(), 
                                             cpts_w_claims_fin['avg_hcp_cost_y'],
                                             cpts_w_claims_fin['avg_hcp_cost_x'])

In [167]:
## Drop unnecessary columns
cpts_w_claims_fin.drop(columns=['avg_hcp_cost_y', 'sd_hcp_cost'], inplace=True)

In [168]:
## Rename "_x" to the normal name - 'avg_hcp_cost'
cpts_w_claims_fin.rename(index=str, columns={'avg_hcp_cost_x': 'avg_hcp_cost'}, inplace=True)

In [169]:
## To ensure calculations are defined, replace nulls with 0.
cpts_w_claims_fin['UNITS_man'].fillna(0, inplace = True)

## Calculate ROI

For each spec/cpt code pair, we want to comapre the cost of reviewing the pair with the sum of dollars denied through that review. This is the "ROI" of reviewing. In cases where sum of denied dollars is greater than the cost of review, we recommend NOT auto-approving and continue to review. In cases where sum of denied dollars is less than the cost of review then we recommend auto-approving it. 

In [170]:
## Calculate the total dollars denied for a pair
cpts_w_claims_fin['sum_cost_denied'] = cpts_w_claims_fin['is_den']*cpts_w_claims_fin['UNITS_man']*cpts_w_claims_fin['avg_hcp_cost']

In [171]:
## Calc ROI for a pair
cpts_w_claims_fin['ROI'] = cpts_w_claims_fin['sum_cost_denied']/cpts_w_claims_fin['cost_to_review']

In [172]:
## For groups where we don't know the average cost from 2018, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 100 so they are NOT included in the dictionaries to auto-approve going forward.
cpts_w_claims_fin['ROI'] = np.where(cpts_w_claims_fin['avg_hcp_cost'].isnull(), 100, cpts_w_claims_fin['ROI'])

In [173]:
## For groups that were auto-approved at 100%, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 0 so they are included in the dictionaries to auto-approve going forward.
cpts_w_claims_fin['ROI'] = np.where(cpts_w_claims_fin['ROI'].isnull(), 0.01, cpts_w_claims_fin['ROI'])
        

In [174]:
## Create a flag based on ROI indicated whether the analytics recommend a pair to be auto-approved
cpts_w_claims_fin['fin_aa_rec'] = np.where(cpts_w_claims_fin['ROI']<1, 1, 0)

In [175]:
## QA step - Check that this equals G&A
cpts_w_claims_fin['cost_to_review'].sum() == dept_ga

True

In [176]:
cpts_w_claims_fin_last3 = cpts_w_claims_fin[cpts_w_claims_fin['last3'] == 1]

In [177]:
cpts_w_claims_fin_prior3 = cpts_w_claims_fin[cpts_w_claims_fin['last3'] == 0]

In [178]:
cpts_w_claims_fin_prior3.columns

Index(['last3', 'Specialty', 'CPT_Code', 'is_PPL', 'UNITS', 'is_autoapp',
       'UNITS_man', 'is_den', 'cost_to_review', 'cnt_hcp_cost', 'avg_hcp_cost',
       'sum_cost_denied', 'ROI', 'fin_aa_rec'],
      dtype='object')

In [179]:
cpts10 = pd.merge(cpts_w_claims_fin_last3, cpts_w_claims_fin_prior3, on = ['Specialty', 'CPT_Code', 'is_PPL'], 
                                   suffixes=('', '_prior3'), how='outer')

In [180]:
cpts10.drop(['last3', 'cnt_hcp_cost', 'cost_to_review', 'sum_cost_denied', 'last3_prior3', 'cnt_hcp_cost_prior3',
                               'cost_to_review_prior3', 'cnt_hcp_cost_prior3', 'avg_hcp_cost_prior3', 'sum_cost_denied_prior3'], axis=1
                              , inplace=True)

In [181]:
## Calculate Increase in volume
cpts10['vol_increase'] = (cpts10['UNITS']/cpts10['UNITS_prior3']) - 1

In [182]:
## Calculate increase in AA rate
cpts10['AA_rate_increase'] = cpts10['is_autoapp']-cpts10['is_autoapp_prior3']

In [183]:
## Calculate increase in denial rate
cpts10['den_rate_increase'] = cpts10['is_den']-cpts10['is_den_prior3']

In [184]:
cpts10 = cpts10.merge(cpt_desc, how='inner', on = 'CPT_Code')

In [185]:
cpts11 = pd.merge(clinical_decisions_current, cpts10, on=['Specialty', 'CPT_Code', 'is_PPL'], how = 'outer')

In [186]:
cpts11.columns

Index(['Specialty', 'CPT_Code', 'is_PPL', 'Decision_Source', 'Decision_Maker',
       'Reason', 'Decision_Date', 'Decision', 'UNITS', 'is_autoapp',
       'UNITS_man', 'is_den', 'avg_hcp_cost', 'ROI', 'fin_aa_rec',
       'UNITS_prior3', 'is_autoapp_prior3', 'UNITS_man_prior3',
       'is_den_prior3', 'ROI_prior3', 'fin_aa_rec_prior3', 'vol_increase',
       'AA_rate_increase', 'den_rate_increase', 'Name'],
      dtype='object')

In [187]:
## Reorder the columns to make easier to compare

cpts11 = cpts11[['Specialty', 'CPT_Code', 'Name', 'is_PPL', 'Decision_Source',
       'Reason', 'Decision_Maker', 'Decision_Date', 'Decision', 'UNITS',
       'UNITS_prior3', 'vol_increase', 'is_autoapp', 'is_autoapp_prior3',
       'AA_rate_increase', 'is_den', 'is_den_prior3', 'den_rate_increase',
       'UNITS_man', 'UNITS_man_prior3', 'avg_hcp_cost', 'ROI', 'ROI_prior3',
       'fin_aa_rec', 'fin_aa_rec_prior3']]

In [188]:
cpts11.to_csv('../Data/check.csv')

In [189]:
cpts11.head()

,Specialty,CPT_Code,Name,is_PPL,Decision_Source,Reason,Decision_Maker,Decision_Date,Decision,UNITS,UNITS_prior3,vol_increase,is_autoapp,is_autoapp_prior3,AA_rate_increase,is_den,is_den_prior3,den_rate_increase,UNITS_man,UNITS_man_prior3,avg_hcp_cost,ROI,ROI_prior3,fin_aa_rec,fin_aa_rec_prior3
0,OPHTHALMOLOGY,J2001,NaN,0,Rule,Pend EPL,NaN,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FACILITY SERVICES,Z7610,MISC DRUGS AND MED SUPPLI,1,Rule,Pend Specialty,UM Team,2019-02-01,0.0,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,0.0523,0.0,NaN,1.0,NaN
2,INTERNAL MEDICINE,83615,NaN,1,Rule,Low Volume,Pioneer,2019-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTERVENTIONAL RADIOLOGY,J2250,NaN,1,Rule,Low Volume,Pioneer,2019-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FAMILY PRACTICE,J7620,ALBUTEROL TO 2.5 MG & IPR,0,Rule,Pend EPL,NaN,NaT,0.0,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,0.1493,0.0,NaN,1.0,NaN


In [190]:
cpts11[cpts11['CPT_Code']=='99215']

,Specialty,CPT_Code,Name,is_PPL,Decision_Source,Reason,Decision_Maker,Decision_Date,Decision,UNITS,UNITS_prior3,vol_increase,is_autoapp,is_autoapp_prior3,AA_rate_increase,is_den,is_den_prior3,den_rate_increase,UNITS_man,UNITS_man_prior3,avg_hcp_cost,ROI,ROI_prior3,fin_aa_rec,fin_aa_rec_prior3
1861,GENERAL PRACTICE,99215,NaN,0,Rule,Pend EPL,NaN,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1938,FAMILY PRACTICE,99215,EST PT OFFICE VISIT - COM,0,Rule,Pend EPL,NaN,NaT,0.0,3.0,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,3.0,137.078900,0.000000,0.000000,1.0,1.0
1985,EMERGENCY SERVICES,99215,NaN,0,Rule,Pend EPL,NaN,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8400,INTERVENTIONAL RADIOLOGY,99215,NaN,1,Rule,Low Volume,Pioneer,2019-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9645,INTERNAL MEDICINE,99215,NaN,1,Rule,Low Volume,Pioneer,2019-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9703,GENERAL PRACTICE,99215,NaN,1,Rule,Low Volume,Pioneer,2019-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9764,WOUND CARE,99215,EST PT OFFICE VISIT - COM,1,Rule,Low Volume,Pioneer,2019-02-01,0.0,NaN,1.0,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,1.0,NaN,NaN,0.000000,NaN,1.0
9856,PSYCHIATRY,99215,NaN,1,Rule,Low Volume,Pioneer,2019-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,NEUROPSYCHIATRY,99215,NaN,0,Rule,Pend EPL,NaN,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10105,ONCOLOGY - MEDICAL,99215,NaN,1,Rule,Low Volume,Pioneer,2019-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create Dictionaries

In [191]:
## Model Additional Auto Approvals when different thresholds are set.
## Approach: use "given" threshold to determine which CPT codes are "auto-approve"-able for each specialty
##  - For loop through referrals, return 1 if all CPT codes are on "auto-approve"-able list, else 0 

In [192]:
list_o_types_to_pend = ['INPT ADM', 'EMERGENCY ROOM', 'DAY SURG', 'OOA INPT', 
                        'SKILLED NURSING', 'OBSERVATION', 'OB OBSERVATION']

In [193]:
cpts_w_claims_fin.reset_index(drop=True, inplace=True)

In [194]:
## A function that uses cpt_w_claims_fin to generate a dictionary codes to auto-approve.
def collect_clinical_decisions(specialty_cpt, list_to_change, spec_list, rsn, new_status=0):
    decision_sources = cpts_w_claims_fin['dec_source'].to_list()
    overrule_reasons = cpts_w_claims_fin['overrule_rsn'].to_list()
    final_decisions = cpts_w_claims_fin['final_decision'].to_list()
    for index, row in specialty_cpt.iterrows():
        if row['Specialty'] in spec_list:
            if row['CPT_Code'] in list_to_change:
                decision_sources[index] = 'clinical'
                overrule_reasons[index] = rsn
                final_decisions[index] = new_status
    specialty_cpt['dec_source'] = decision_sources
    specialty_cpt['overrule_rsn'] = overrule_reasons
    specialty_cpt['final_decision'] = final_decisions
    return specialty_cpt
    

In [195]:
## Create new column in cpt_w_claims_fin that will store decision source.
cpts_w_claims_fin['dec_source'] = 'DBA'

In [196]:
## Create new column in cpt_w_claims_fin that will store clinical overrule_rsn
cpts_w_claims_fin['overrule_rsn'] = np.nan

In [197]:
## Create new column in cpt_w_claims_fin that will store final decision
cpts_w_claims_fin['final_decision'] = cpts_w_claims_fin['fin_aa_rec']

In [198]:
## Ensure that the dictionary for EPL is empty, meaning none will auto-approve
spec_dict_EPL = {k: [] for k in list_o_specs}

In [199]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='DERMATOLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

258

## Define codes to ADD or REMOVE from each list

The general process for creating dictionaries is to do the g&a analysis first.  These are noted with "DBA" (Decision by Analysis.

Next, in chronological order, we apply clinical decisions. In the first review session (Q2 2019) this was the order:
1. A list of PRL codes that Chris McRae said should be auto-approved
2. A list of PRL codes that Chris McRae said should be pended - "codestoremove"
3. Pend Specialties that should be pended categorically
4. Laurie and Chris's first round of mostly pends.  Rhoda also took a pass on 5/3/2019
5. Phyllis and Rhoda pass at pended specialties. 5/17/2019
6. Remove low volume codes.
7. Pend all EPL Codes

In [200]:
## ADD Derm codes - provided my Christel McRae (124)
derm_add0 = ['10060','10061','11000','11055','11057','11100','11101','11300','11301','11302','11303','11305',
            '11306','11307','11308','11310','11311','11312','11313','11404','11426','11440','11600',
            '11601','11602','11603','11604','11606','11620','11621','11622','11623','11624','11626','11640',
            '11641','11642','11643','11644','11646','11900','11901','29580','54050','54056','54100','67810',
            '69100','87101','87220','J3301','10061','10080','10081','10140','10180','11055','11056','11057',
            '11101','11300','11301','11302','11303','11305','11306','11307','11308','11310','11311','11312',
            '11313','11404','11426','11440','11444','11446','11600','11601','11602','11603','11604',
            '11606','11620','11621','11622','11623','11624','11626','11640','11641','11642','11643','11644',
            '11646','11900','11901','54105']


In [201]:
## find unique values (72)
derm_add = list(set(derm_add0))

In [202]:
## ADD Pain Add for PPL
pain_add0 = ['99203','99204','99213','99214']

In [203]:
## ADD Gastro and General Surgery for PPL and EPL
gi_add0 = ['45378','45380','45385','G0105','G0121','00812']

In [204]:
## ADD Blood Transfusions - across all specialties PPL and EPL
bt_add0 = ['36430','86900','86901','86902','86903','86904','86905','86906','86907','86908','86909','86910'
          ,'86911','86912','86913','86914','86915','86916','86917','86918','86919','86920']

In [205]:
## ADD Podiatry
podiatry_add0 = ['11055']

In [206]:
## REMOVE from all dictionaries
codes_to_remove0 = ['97810','95115','95116','95117','95170','95180','95181','95182','95183','95184','95185','95186',
                   '95187','95188','95199','95004','A0999','A0426','A0427','A0428','A0429','92590','92591','92592',
                   '92593','92594','90901','90902','90903','90904','90905','90906','90907','90908','90909','90910',
                   '90911','90875','90876','86890','93798','98940','J1050','58301','58565','58611','G0337',
                   '74740','89320','J3490','J0717','J1438','J7321','J7322','J7323','J7324','J7325','J3489','J0135',
                   'J3030','J1830','J9215','J9218','J0129','J9202','J3301','J1745','96365','96366','90378','J2505',
                   'J1440','J1441','J0885','J0886','T1013','96118','96119','96120','97166','97167','97168','97110',
                   '92015','97162','97163','97110','36468','92506','55250','59840',
                   'S0199','S0618','92595','58300','J7300','57170','A4266','11981',
                   'J7302','J7300','58670','59600','58605','64612','J0585','11055',
                   '11057','11719','11721','G0127','S0390']

In [207]:
codes_to_remove = list(set(codes_to_remove0))

In [208]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='GASTROENTEROLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

182

In [209]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='DERMATOLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

258

In [210]:
cpts_w_claims_fin.pivot_table(values='CPT_Code', index=['final_decision'], aggfunc='count', margins=True)

,CPT_Code
final_decision,
0,5634
1,34268
All,39902


#### Add specific codes to specific specialties

In [211]:
## List of codes to auto-approve add derm
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, derm_add, ['DERMATOLOGY'], rsn='from PRL', new_status=1)

In [212]:
## List of codes to auto-approve add pain
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, pain_add0, ['PAIN MANAGEMENT'], rsn='from PRL', new_status=1)

In [213]:
## List of codes to auto-approve add gastro
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, gi_add0, 
                                               ['GASTROENTEROLOGY','SURGERY - GENERAL'], rsn='from PRL', new_status=1)

In [214]:
## List of codes to auto-approve add blood transfusions
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, bt_add0, list_o_specs, rsn='from PRL', new_status=1)

In [215]:
## List of codes to auto-approve add podiatry
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, podiatry_add0, ['PODIATRY'], rsn='from PRL', new_status=1)

#### Pend specific set of codes across specialties

In [216]:
## List of codes to pend across specialties
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, codes_to_remove, list_o_specs, rsn='from PRL', new_status=0)

#### Soft Pend Specialties categorically

In [217]:
## Specialties for which the clinical team prefers to ignore the "decision by analysis" recommendation and instead
## choose a few codes to AA, but mostly the specialty will pend
## To ensure that no referrals are auto-approved from it
specs_that_should_pend = ['ACUPUNCTURE', 'ADDICTION MEDICINE', 'ANESTHESIOLOGY',
                         'BEHAVIORAL HEALTH', 'CHIROPRACTIC', 'DENTIST', 'DME MAINTENANCE',
                         'LICENSED CLIN SOCIAL WORKER', 'MFCC (THERAPIST)', 'NON-CONTRACT UNKWN BILL AREA',
                         'NURSE PRACTITIONER', 'NURSING FACILITY - OTHER', 'OPTICIAN',
                        'PEDS-DEVELOPMENTAL BEHAVIORAL', 'PSYCHIATRY', 'PSYCHOLOGY', 'SENIOR WELLNESS VISIT',
                         'SPORTS MEDICINE', 'SURGERY - ORAL', 'AMBULATORY SURGICAL CENTER', 'AMBULANCE', 'ALLERGY/IMMUNOLOGY',
                         'CUSTODIAL CARE', 'HOME HEALTH', 'INFERTILITY', 'OPTOMETRY', 'PALLIATIVE CARE', 'PODIATRY', 'SLEEP STUDY',
                         'SNF - FAC', 'OCCUPATIONAL THERAPY', 'PHYSICAL THERAPY/REHAB', 'FACILITY SERVICES',
                         'GENETICS', 'INTERVENTIONAL RADIOLOGY', 'LABORATROY', 'NUCLEAR MEDICINE', 'ONCOLOGY - GYN',
                         'PATHOLOGY', 'PHARMACY', 'SURGERY - HAND', 'SURGERY - CARDIAC', 'SURGERY - MAXILLOFACIAL ORAL',
                         'SURGERY - PLASTIC/RECONSTRUCT']

In [218]:
for spec in specs_that_should_pend:
    cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Specialty'] == spec, 
                                    'Soft Pend Specialty', cpts_w_claims_fin['overrule_rsn'])
    cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    'clinical', cpts_w_claims_fin['dec_source'])
    cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    0, cpts_w_claims_fin['final_decision'])

In [219]:
def add_clinical_decisions(cpts_w_claims_fin, clinical_decisions, drop_columns = ['Decision', 'Notes', 'overturned']):
    cpts_w_claims_fin = cpts_w_claims_fin.merge(clinical_decisions, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])
    cpts_w_claims_fin['Decision'] = np.where(cpts_w_claims_fin['Decision'].isna(), cpts_w_claims_fin['final_decision'], cpts_w_claims_fin['Decision'])
    cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Notes'], cpts_w_claims_fin['overrule_rsn'])
    cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    'clinical', cpts_w_claims_fin['dec_source'])
    cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Decision'], cpts_w_claims_fin['final_decision'])
    cpts_w_claims_fin.drop(drop_columns, axis=1, inplace=True)
    return cpts_w_claims_fin

#### Remove codes chosen by Laurie and Christel 5/03/2019

In [220]:
#cpts_w_claims_fin = add_clinical_decisions(cpts_w_claims_fin, clinical_decisions_20190503_remove)

#### Add codes chosen by Laurie and Christel 5/03/2019

In [221]:
#cpts_w_claims_fin = add_clinical_decisions(cpts_w_claims_fin, clinical_decisions_20190503_add)

#### Add codes chosen by Phyllis and Rhoda 5/17/2019

In [222]:
#cpts_w_claims_fin = add_clinical_decisions(cpts_w_claims_fin, clinical_decisions_20190517)

#### codes chosen by Laurie and Rhoda 5/22/2019

In [223]:
#cpts_w_claims_fin = add_clinical_decisions(cpts_w_claims_fin, clinical_decisions_20190610, ['Decision', 'Notes'])

In [224]:
cpts_w_claims_fin.columns

Index(['last3', 'Specialty', 'CPT_Code', 'is_PPL', 'UNITS', 'is_autoapp',
       'UNITS_man', 'is_den', 'cost_to_review', 'cnt_hcp_cost', 'avg_hcp_cost',
       'sum_cost_denied', 'ROI', 'fin_aa_rec', 'dec_source', 'overrule_rsn',
       'final_decision'],
      dtype='object')

In [225]:
clinical_decisions_20190610.columns

NameError: name 'clinical_decisions_20190610' is not defined

In [ ]:
cpts_w_claims_fin = add_clinical_decisions(cpts_w_claims_fin, clinical_decisions_20190610, ['Decision', 'Notes', 'Decision_Date',
                                                                                           'Decision_Source', 'Decision_Maker'])

#### Pend codes with fewer than 30 in the past year

In [ ]:
## Pend codes that have volume < 30
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    'low volume', cpts_w_claims_fin['overrule_rsn'])

In [ ]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    'rule', cpts_w_claims_fin['dec_source'])

In [ ]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    0, cpts_w_claims_fin['final_decision'])

#### Hard Pend Specialties categorically

In [ ]:
## Specialties that the clinical team wants to pend categorically

specs_that_should_pend = ['ACUPUNCTURE', 'ADDICTION MEDICINE', 'ANESTHESIOLOGY',
                         'BEHAVIORAL HEALTH', 'CHIROPRACTIC', 'DENTIST', 'DME MAINTENANCE',
                         'LICENSED CLIN SOCIAL WORKER', 'MFCC (THERAPIST)', 'NON-CONTRACT UNKWN BILL AREA',
                         'NURSE PRACTITIONER', 'NURSING FACILITY - OTHER', 'OPTICIAN',
                        'PEDS-DEVELOPMENTAL BEHAVIORAL', 'PSYCHIATRY', 'PSYCHOLOGY', 'SENIOR WELLNESS VISIT',
                         'SPORTS MEDICINE', 'SURGERY - ORAL', 'AMBULATORY SURGICAL CENTER', 'AMBULANCE',
                         'CUSTODIAL CARE', 'HOME HEALTH', 'INFERTILITY', 'OPTOMETRY', 'PALLIATIVE CARE', 'SLEEP STUDY',
                         'SNF - FAC', 'OCCUPATIONAL THERAPY', 'NUCLEAR MEDICINE',
                         'PATHOLOGY', 'PHARMACY', 'SURGERY - HAND']

In [ ]:
for spec in specs_that_should_pend:
    cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Specialty'] == spec, 
                                    'Hard Pend Specialty', cpts_w_claims_fin['overrule_rsn'])
    cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    'rule', cpts_w_claims_fin['dec_source'])
    cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    0, cpts_w_claims_fin['final_decision'])

#### Pend all EPL

In [ ]:
## Pend EPL
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    'Pend EPL', cpts_w_claims_fin['overrule_rsn'])

In [ ]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    'rule', cpts_w_claims_fin['dec_source'])

In [ ]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    0, cpts_w_claims_fin['final_decision'])

In [ ]:
## Function that gathers CPT codes that should be pended and stores them in a dictionary.
def create_dict_of_CPT_codes_v3(specialty_cpt, list_o_specs, PPL=1):
    spec_dict = {k: [] for k in list_o_specs}
    for index, row in specialty_cpt.iterrows():
        if row['final_decision'] == 1:
            if row['is_PPL'] == PPL:
                spec_dict[row['Specialty']].append(row['CPT_Code'])
    return spec_dict

In [ ]:
spec_dict_PPL = create_dict_of_CPT_codes_v3(cpts_w_claims_fin, list_o_specs, PPL=1)

In [ ]:
print(spec_dict_PPL['ENDOCRINOLOGY'])

In [ ]:
#### Prepare a newer set of referrals to run through the new code.

In [ ]:
## Assigns an auto-approve (1) or pend (0) status to each specialty / code group
def assign_status(codes, spec_dict_PPL, spec_dict_EPL, list_of_types_pend):
    status = list(np.zeros(codes.shape[0]))
    specs_list = codes['Specialty'].unique().tolist()
    print(len(status))
    print(codes.shape[0])
    for spec in specs_list:
        if spec not in spec_dict_PPL:
            spec_dict_PPL[spec] = []
    for spec in specs_list:
        if spec not in spec_dict_EPL:
            spec_dict_EPL[spec] = []
    for index, row in codes.iterrows():
        if row['is_PPL'] == 1:
            if row['CPT_Code'] in spec_dict_PPL[row['Specialty']]:
                if row['ref_type'] not in list_of_types_pend:
                    status[index] = 1
        ##else:
          ##  if row['CPT_Code'] in spec_dict_EPL[row['Specialty']]:
            ##    status[index] = 0
    return status

In [ ]:
cpts.reset_index(drop=True, inplace=True)

In [ ]:
auto_approve = assign_status(cpts, spec_dict_PPL, 
                             spec_dict_EPL, list_o_types_to_pend)

In [ ]:
cpts['auto_approvable'] = auto_approve

In [ ]:
refs_results = cpts.groupby(['HCP_CONNECT_AUTH_NUMBER'], as_index=False).agg({'auto_approvable': 'mean',
                                                                                 'is_den': 'mean'})

In [ ]:
refs_results['aa-yn'] = np.where(refs_results['auto_approvable']==1, 1, 0)

In [ ]:
refs_results.drop('auto_approvable', axis=1, inplace=True)

In [ ]:
refs_results.columns

In [ ]:
refs_results.pivot_table('HCP_CONNECT_AUTH_NUMBER', index='aa-yn', columns='is_den', aggfunc='count', margins=True)

In [ ]:
refs_results.drop('is_den', axis=1, inplace=True)

In [ ]:
cpts = cpts.merge(refs_results, how='left', on=['HCP_CONNECT_AUTH_NUMBER'])

In [ ]:
#cpts.to_csv('../Data/Outputs/expected_golive_2019.csv')

In [ ]:
cpts.pivot_table('HCP_CONNECT_AUTH_NUMBER', index='aa-yn', columns='urgent', aggfunc='count', margins=True)

In [ ]:
cpts_w_claims_fin[cpts_w_claims_fin['CPT_Code']=='J1561'].head()

## Region Switchpoint

In [ ]:
## Use this code to conduct the results portion of the code for a single region
#cpts = cpts[cpts['region']=='SAN GABRIEL VALLEY']

In [ ]:
cpts.pivot_table(values='UNITS', index=['is_autoapp', 'is_den'], columns='aa-yn', aggfunc='count', margins=True)##.to_csv('../Data/Outputs/costs_cm.csv')

## Construct Specialty Summary

In [ ]:
refs_new_head = cpts.groupby(['Specialty', 'ref_type', 'HCP_CONNECT_AUTH_NUMBER'], as_index=False).agg({
    'UNITS': 'count',
    'is_autoapp': 'mean',
    'is_PPL': 'mean',
    'is_app': 'mean',
    'is_den': 'mean',
    'aa-yn': 'mean'
})

In [ ]:
refs_new_head.pivot_table(values=['is_autoapp', 'aa-yn'], index='Specialty', aggfunc=['count', 'mean']).to_csv('../Data/Outputs/spec_rates.csv',
                                                                                                              sep='|')

In [ ]:
specialty_summary = pd.read_csv('../Data/Outputs/spec_rates.csv', sep='|', skiprows=3, names=['Specialty',
                                                                                             'ref_vol', 'ref_vol2',
                                                                                             'new_rate', 'old_rate'])

In [ ]:
specialty_summary.drop('ref_vol2', axis=1, inplace=True)

In [ ]:
## Get counts of codes for each specialty

In [ ]:
def spec_code_counts(spec_dict, spec_summary):
    counts = []
    for index, row in spec_summary.iterrows():
        counts.append(len(spec_dict[row['Specialty']]))
    return counts

In [ ]:
PPL_counts = spec_code_counts(spec_dict_PPL, specialty_summary)

In [ ]:
specialty_summary['num_codes']=PPL_counts

In [ ]:
## get 'denied now approved'

In [ ]:
refs_new_head['denied_now_aa'] = refs_new_head['is_den'] * refs_new_head['aa-yn']

In [ ]:
refs_new_head.pivot_table('denied_now_aa', index='Specialty', aggfunc='sum').to_csv('../Data/Outputs/spec_den_now_aa.csv',
                                                                                                      sep='|')

In [ ]:
ss_deny_now_aa = pd.read_csv('../Data/Outputs/spec_den_now_aa.csv', sep='|', skiprows=2, names=['Specialty', 'deny_now_aa'])

In [ ]:
specialty_summary = specialty_summary.merge(ss_deny_now_aa, how='inner', on='Specialty')

In [ ]:
## Prep cpts_w_claims_fin to merge into cpts

In [ ]:
cpts_w_claims_fin_select = cpts_w_claims_fin.drop(['UNITS', 'is_autoapp', 'is_den',
       'UNITS_man', 'cost_to_review', 'cnt_hcp_cost',
       'sum_cost_denied'], axis=1)

In [ ]:
cpts_w_fin = pd.merge(cpts, cpts_w_claims_fin_select, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [ ]:
cpts_w_fin['dollars_denied_now_app'] = cpts_w_fin['is_den']*cpts_w_fin['aa-yn']*cpts_w_fin['avg_hcp_cost']

In [ ]:
cpts_w_fin['cost_to_review_bene'] = (1-cpts_w_fin['is_autoapp'])*cpts_w_fin['aa-yn']*ga_cpt

In [ ]:
cpts_w_fin['cost_to_review_loss'] = cpts_w_fin['is_autoapp']*(1-cpts_w_fin['aa-yn'])*ga_cpt

In [ ]:
cpts_w_fin['new_denial_bene_est'] = cpts_w_fin['is_autoapp']*(1-cpts_w_fin['aa-yn'])*ga_cpt*cpts_w_fin['ROI']*cpts_w_fin['is_PPL']

In [ ]:
cpts_w_fin.pivot_table(values=['dollars_denied_now_app', 'cost_to_review_bene', 'cost_to_review_loss',
                                  'new_denial_bene_est'], index='Specialty', aggfunc='sum').to_csv('../Data/Outputs/specialty_fins.csv', sep='|')

In [ ]:
specialty_fins = pd.read_csv('../Data/Outputs/specialty_fins.csv', sep='|')

In [ ]:
specialty_summary = specialty_summary.merge(specialty_fins, how='inner', on='Specialty')

In [ ]:
specialty_summary['net_benefit'] = specialty_summary['cost_to_review_bene'] - specialty_summary['cost_to_review_loss'] - specialty_summary['dollars_denied_now_app']

In [ ]:
specialty_summary['net_benefit_w_new_den'] = specialty_summary['cost_to_review_bene'] - specialty_summary['cost_to_review_loss'] -specialty_summary['dollars_denied_now_app'] + specialty_summary['new_denial_bene_est']

In [ ]:
# Give the filename you wish to save the file to
spec_summary_filename = '../Data/Outputs/specialty_summary_fin.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(spec_summary_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    specialty_summary.to_excel(spec_summary_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'volume', 'new rate', 'old_rate', '# codes in dictionary', 'false positives',
                                 'cost to review, benefit', 'cost to review, loss', 'dollars denied, now approved',
                                 'new denials, benefit (est)', 'new benefit', 'net benefit, est'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')

In [ ]:
cpts_w_claims_fin['overrule_rsn'].unique()

In [ ]:
refs_results['aa-yn'].mean()

In [ ]:
## rate before laurie and chrisel's codes to pend
v0 = 1198038/2086468
print(v0)

In [ ]:
## rate after laurie and christels' codes to pend
v1 = 1091109/2086468
print(v1)

In [ ]:
v2 = 1097619/2086468
print(v2)

In [ ]:
## using 2018 data to create rules, and then applying them to early 2019 data
v3 = 274250/543258
print(v3)

In [ ]:
## using clinical review list from 20190501 from Rhoda 
.401

In [ ]:
## 20190503 V1 - More Conservative Version
.4532

In [ ]:
## 20190503 v2 - Baseline
.462

In [ ]:
## 20190517 - after Phyllis and Rhoda add back several codes
.52

In [ ]:
## 20190524
.525

## Code Detail List

In [ ]:
spec_cpt_w_projected_aa_rate = cpts.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'aa-yn': 'mean'
})

In [ ]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(spec_cpt_w_projected_aa_rate, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [ ]:
cpts['UNITS'].sum()

In [ ]:
# Give the filename you wish to save the file to
spec_cpt_w_projected_aa_rate_fin_filename = '../Data/Outputs/spec_cpt_w_projected_aa_rate.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(spec_cpt_w_projected_aa_rate_fin_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    spec_cpt_w_projected_aa_rate.to_excel(spec_cpt_w_projected_aa_rate_fin_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'CPT Code', 'PPL?', 'new rate'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')

In [ ]:
cpts.pivot_table('CPT_Code', index='aa-yn', columns='is_den', aggfunc='count', margins=True)

In [ ]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(cpt_desc, how='inner', on = 'CPT_Code')

In [ ]:
cpts_w_claims_fin[cpts_w_claims_fin['Specialty']=='CARDIOLOGY']

In [ ]:
# Give the filename you wish to save the file to
cpts_w_claims_fin_filename = '../Data/Outputs/spec_cpt_summary.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(cpts_w_claims_fin_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    cpts_w_claims_fin.to_excel(cpts_w_claims_fin_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'CPT Code', 'PPL?', 'volume', 'old rate', 'denial rate',
       'volume, reviewed', 'cost to review', 'avg. cost', 'denied dollars', 'ROI',
       'decision by ROI', 'decision source', 'clinical reason', 'final decision', 'new rate',
       'CPT desc'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')

In [ ]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='SURGERY - CARDIAC') &
                 (cpts_w_claims_fin['final_decision']==1)]

## Generate Rule Spreadsheet that can be uploaded to CCT

CCT, the tool that houses the referrals auto-approval rules and processing, can auto-generate rules when provided a spreadsheet of a specific format.  This portion of the code reads in that file, and then completes it with the information required.

#### Read in file to get accurate headers and format

In [ ]:
rule_file = pd.read_excel('../Data/RMS_Rules.xlsx', sheet_name='RULE_AUTOAPPROVEPX')

In [ ]:
rule_file.columns

In [ ]:
## Add codes to dictionary for specific specialties that appear in CCT data, but not the data warehouse

In [ ]:
spec_dict_PPL['HIV/AIDS SPECIALIST'] = ['99203', '99204', '99213', '99214']

In [ ]:
spec_dict_PPL

In [ ]:
def create_rule_excel_file(rule_file, spec_dict, list_o_specs, PPL=True):
    rule_file['Auth_Status'] = 8
    rule_file['Auth_Status_Num'] = 1
    rule_file['RefType_Key'] = np.nan
    rule_file['RefType_Num'] = np.nan
    rule_file['ByPass_Adjudication'] = False
    rule_file['Rule_Priority'] = 2000
    rule_file['Rule_Category'] = 2
    rule_file['Eff_Date'] = pd.datetime(2019, 6, 15)
    rule_file['Term_Date'] = np.nan
    rule_file['Entity_List'] = 'All'
    rule_file['Site_List'] = 'All'
    rule_file['HMO_List'] = 'All'
    rule_file['Product_Type_List'] = 'All'
    rule_file['Dx_List'] = np.nan
    rule_file['Age_From'] = 18
    rule_file['Age_To'] = 199
    rule_file['Gender'] = 'A'
    rule_file['GoldStar'] = 'A'
    rule_file['POS_List'] = 'All'
    rule_file['RefTo_Provider_List'] = 'All'
    rule_file['RefFrom_Spc_List'] = 'All'
    rule_file['Facility_Spc_List'] = 'All'
    rule_file['Spc_Operator'] = 'OR'
    rule_file['Date_Added'] = pd.datetime(2019, 6, 14)
    rule_file['Added_By'] = 'PIONEER'
    rule_file['Archive_Flag'] = False
    rule_file['RefFrom_Provider_List'] = 'ALL'
    rule_file['RefTo_Status_List'] = 'PPL'
    rule_key = list(rule_file['Rule_Key'])
    Px_list = list(rule_file['Px_List'])
    rule_code = list(rule_file['Rule_Code'])
    rule_desc = list(rule_file['Rule_Desc'])
    rule_lob = list(rule_file['LOB_List'])
    ref_to_spec = list(rule_file['RefTo_Spc_List'])
    index = 0
    for spec in list_o_specs:
        if len(spec_dict[spec]) == 0:
            continue
        else:
            rule_key[index] = index+1
            Px_list[index] = spec_dict[spec]
            rule_code[index] = (spec + '_PPL').replace(' ', '')
            rule_desc[index] = ('auto-approve ' + spec + ' referrals to PPL providers for specific codes.')
            if spec[-7:] == 'not_sen':
                print('not_sne')
                print(spec[:-8])
                ref_to_spec[index] = spec[:-8]
            elif spec[-6] == 'senior':
                ref_to_spec[index] = spec[:-7]
            else:
                ref_to_spec[index] = spec
            if spec[-6:]=='senior':
                rule_lob[index] = 'SENIOR'
            else:
                rule_lob[index] = 'All'
            index+=1
    rule_file['Rule_Key'] = rule_key
    rule_file['Px_List'] = Px_list
    rule_file['Rule_Desc'] = rule_desc
    rule_file['Rule_Code'] = rule_code
    rule_file['LOB_List'] = rule_lob
    rule_file['RefTo_Spc_List'] = ref_to_spec
    rule_file.drop(rule_file.index[index:], inplace=True)
    return rule_file

In [ ]:
'RADIOLOGY_not_sen'[-7:] in ('not_sen', 'senior')

In [ ]:
'RADIOLOGY_senior'[:-7]

In [ ]:
new_rule_file = create_rule_excel_file(rule_file, spec_dict_PPL, list_o_specs)

In [ ]:
new_rule_file['RefTo_Spc_List']

In [ ]:
# Give the filename you wish to save the file to
rules_to_upload_filename = '../Data/Outputs/RMS_Rules_new.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(rules_to_upload_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    new_rule_file.to_excel(rules_to_upload_filename, index=False, )
else:
    print('WARNING: This file already exists!')